# NLP: Analyzing Healthcare Reviews and Predicting Their Useful Votes




In [1]:
import os
import codecs
import json
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

data_directory = os.path.join('../data')
businesses_filepath = os.path.join(data_directory, 'yelp_academic_dataset_business.json')
review_json_filepath = os.path.join(data_directory, 'yelp_academic_dataset_review.json')
intermediate_directory = os.path.join(data_directory, 'intermediate')
review_txt_filepath = os.path.join(intermediate_directory, 'review_text_all.txt')

In [2]:
healthcare_ids = []

# open the businesses file
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business has no categories or is not a target entity, skip to the next one
        if business[u'categories'] is None or u'Health & Medical' not in business[u'categories']:
            continue
        # Remove businesses in BW, Germany
        if u'BW' in business[u'state']:
            continue
        # Remove businesses that are restaurants, food and pets
        if u'Restaurants' in business[u'categories'] or u'Food' in business[u'categories'] or 'Pets' in business[u'categories']:
            continue
            
        # add the business id to our healthcare_ids set
        healthcare_ids.append(business[u'business_id'])

# Turn the list of ids into a set, which is faster for testing whether an element is in the set
healthcare_ids = set(healthcare_ids)

# print the number of unique ids in the dataset
print ('{:,}'.format(len(healthcare_ids)), u'health & medical entities in the dataset.')

10,211 health & medical entities in the dataset.


In [3]:
# Create a new file that contains only the text from reviews about healthcare entities.
# One review per line in the this new file.
    
review_count = 0
useful = []
stars = []

# create & open a new file in write mode
with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

    # open the existing review json file
    with codecs.open(review_json_filepath, encoding='utf_8') as review_json_file:

        # loop through all reviews in the existing file and convert to dict
        for review_json in review_json_file:
            review = json.loads(review_json)

            # if this review is not in the target set, skip to the next one
            if review[u'business_id'] not in healthcare_ids:
                continue

            # write each review as a line in the new file
            # escape newline characters in the original review text
            if review[u'text'] is None:
                print(review_count)
            
            review_txt_file.write(review[u'text'].replace('\n', '\\n').replace('\r','') + '\n')
            review_count += 1
            useful.append(review[u'useful'])
            stars.append(review[u'stars'])

print (u'Text from {:,} healthcare reviews written to the new txt file.'.format(review_count))

Text from 114,556 healthcare reviews written to the new txt file.


### Preprocessing the Text

I use spaCy to tokenize, lemmitize and remove stopwords from the text. Apply phrase modeling by looking for words that tend to appear one after another more frequently than by random chance.

In [4]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

In [5]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

unigram_sentences_filepath = os.path.join(intermediate_directory, 'unigram_sentences_all.txt')
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')
bigram_sentences_filepath = os.path.join(intermediate_directory, 'bigram_sentences_all.txt')
trigram_model_filepath = os.path.join(intermediate_directory, 'trigram_model_all')
trigram_sentences_filepath = os.path.join(intermediate_directory, 'trigram_sentences_all.txt')
trigram_reviews_filepath = os.path.join(intermediate_directory, 'trigram_transformed_reviews_all.txt')

In [6]:
def punct_space(token):
    """Eliminate tokens that are pure punctuation or white space"""
    
    return token.is_punct or token.is_space

def person(token):
    """Remove tokens that are PERSON entities"""
    
    return token.ent_type_ == 'PERSON'

def line_review(filename):
    """Generator function (iterator without storing all texts)
    to read in reviews from file and return the original line breaks"""
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')

def lemmatized_sentence_corpus(filename):
    """Generator function to use spaCy to parse reviews, lemmatize the text and yield sentences"""
    
    for parsed_review in nlp.pipe(line_review(filename), batch_size=10000, n_threads=4):
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent 
                             if not (punct_space(token) or person(token))])

In [7]:
# Segment reviews into sentences and normalize the text
# Save the parsed sentences file on disk to avoid storing the entire corpus in RAM
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        f.write(sentence + '\n')

In [8]:
# gensim's LineSentence class takes the format: one sentence = one line
# words are preprocessed and separated by whitespace.
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [9]:
# Run a phrase model to link two-words phrases together
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)
bigram_model = Phrases.load(bigram_model_filepath)

In [10]:
# Apply the bigram model to unigram sentences and create a text with bigram sentences
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf-8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

In [11]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [12]:
trigram_model = Phrases(bigram_sentences)
trigram_model.save(trigram_model_filepath)
trigram_model = Phrases.load(trigram_model_filepath)

In [13]:
with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for bigram_sentence in bigram_sentences:
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
        f.write(trigram_sentence + '\n') 

In [14]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [15]:
# Write a transformed text into a new file, with one review per line
with codecs.open(trigram_reviews_filepath, 'w', encoding='utf-8') as f:
    for parsed_review in nlp.pipe(line_review(review_txt_filepath), batch_size=10000, n_threads=4):

        # Lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review 
                          if not (punct_space(token) or person(token))]

        # Apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # Remove any remaining stopwords
        trigram_review = [term for term in trigram_review 
                          if term not in spacy.en.language_data.STOP_WORDS]

        # Write the transformed review as a new line
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

### Train a Word2Vec Model

In [16]:
from gensim.models import Word2Vec
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

In [17]:
# Train a word2vec model: 100-dimensional word vectors
word2vec = Word2Vec(size=100, window=5, min_count=20, sg=0, workers=4)
word2vec.build_vocab(trigram_sentences) 
word2vec.train(trigram_sentences, total_examples=word2vec.corpus_count, epochs=10)
word2vec.save(word2vec_filepath)

word2vec = Word2Vec.load(word2vec_filepath)

In [18]:
print(u'{:,} terms in the word2vec vocabulary.'.format(len(word2vec.wv.vocab)))

11,251 terms in the word2vec vocabulary.


In [19]:
# build a list of terms, index, and term counts from the word2vec model
ordered_vocab = [(term, vocab.index, vocab.count) for term, vocab in word2vec.wv.vocab.items()]
ordered_vocab = sorted(ordered_vocab, key=lambda ordered_vocab:ordered_vocab[2], reverse=True)
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

In [20]:
# Creat a dictionary mapping each word to a 300-dimensional vector
word_vectors = dict(list(zip(word2vec.wv.index2word, word2vec.wv.syn0)))

In [21]:
word2vec.most_similar(positive=['dentist'], topn=5)

[('pediatric_dentist', 0.7933251261711121),
 ('orthodontist', 0.7528108358383179),
 ('dermatologist', 0.7381713390350342),
 ('pediatrician', 0.735224723815918),
 ('chiropractor', 0.6919575929641724)]

In [22]:
word2vec.most_similar(positive=['dentist', 'baby'], negative=['tooth'],topn=5)

[('ob', 0.6723254919052124),
 ('doctor', 0.639251172542572),
 ('pediatrician', 0.6306763291358948),
 ('doc', 0.6278711557388306),
 ('obgyn', 0.6148397326469421)]

### Predictive Modeling with Document Features Created from Word2Vec 

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso, Ridge
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error, make_scorer
from collections import defaultdict
from tabulate import tabulate

In [25]:
useful = np.array(useful)
luseful = np.log(useful+1)
X = []
with codecs.open(trigram_reviews_filepath, encoding='utf-8') as f:
    for review in f:
        X.append(review)

In [26]:
class MeanEmbeddingVectorizer():
    """ Given a word to vector mapping, vectorize texts 
    by taking the mean of all the word vectors for each document"""
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(list(word2vec.values())[0])
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in review if w in self.word2vec] 
                   or [np.zeros(self.dim)], axis=0) 
                   for review in X])

In [27]:
class TfidfMeanVectorizer():
    """ Weight average word vector features by its tf-idf"""
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(list(word2vec.values())[0])
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x:x)
        tfidf.fit(X)
        # Let an unseem word be as infrequent as the most infreqeunt word
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
        return self
    
    def transform(self, X):
        return np.array([np.mean([self.word2vec[w] * self.word2weight[w]
                                 for w in review if w in self.word2vec] or 
                                [np.zeros(self.dim)], axis=0) 
                         for review in X])   

In [39]:
def cv_rmse(model, X, y, cv=5, scoring='neg_mean_squared_error'):
    """ Compute an overall RMSE across all folds of cross validation"""
    return np.sqrt(np.mean(np.multiply(cross_val_score(
    model, X, y, cv=cv, scoring='neg_mean_squared_error'), -1)))
    
def RMSE(y_true, y_pred):
    """ Root Mean Squared Error"""
    return np.sqrt(np.mean((y_true - y_pred)**2))

def RMSLE(y_true, y_pred):
    """ Root Mean Squared Logarithmic Error"""
    return np.sqrt(np.mean(((np.log(y_true + 1) - np.log(y_pred + 1))**2)))

In [29]:
# Linear Regression and shrinkage methods: Ridge and lasso

lr_w2v = Pipeline([("w2v_vectorizer", MeanEmbeddingVectorizer(word_vectors)), 
                   ("lr", LinearRegression())])
lr_w2v_tfidf = Pipeline([("tfidf_w2v_vectorizer", TfidfMeanVectorizer(word_vectors)),
                        ("lr", LinearRegression())])
ridge_w2v = Pipeline([("w2v_vectorizer", MeanEmbeddingVectorizer(word_vectors)),
                    ("ridge", Ridge(alpha=1))])
ridge_w2v_tfidf = Pipeline([("tfidf_w2v_vectorizer", TfidfMeanVectorizer(word_vectors)),
                    ("ridge", Ridge(alpha=1))])
lasso_w2v = Pipeline([("w2v_vectorizer", MeanEmbeddingVectorizer(word_vectors)), 
                   ("lasso", Lasso(alpha=1))])
lasso_w2v_tfidf = Pipeline([("tfidf_w2v_vectorizer", TfidfMeanVectorizer(word_vectors)),
                        ("lasso", Lasso(alpha=1))])

In [ ]:
w2v_rmse_1 = sorted([(name, cv_rmse(model, X, luseful, cv=5)) 
                      for name, model in [("lr_w2v", lr_w2v), ("lr_w2v_tfidf", lr_w2v_tfidf),
                                         ("lasso_w2v", lasso_w2v), ("lasso_w2v_tfidf", lasso_w2v_tfidf),
                                         ("ridge_w2v", ridge_w2v), ("ridge_w2v_tfidf", ridge_w2v_tfidf)]], 
                  key=lambda x:x[1])

print (tabulate(w2v_rmse_1, floatfmt=".4f", headers=("model", "RMSE_5cv")))

In [30]:
# Ensemble or Tree-based methods: Gradient Boosting, Random Forest, and Extremely Randomized Trees

gbr_w2v = Pipeline([("w2v_vectorizer", MeanEmbeddingVectorizer(word_vectors)),
                    ("gbr", GradientBoostingRegressor(n_estimators=100))])
gbr_w2v_tfidf = Pipeline([("tfidf_w2v_vectorizer", TfidfMeanVectorizer(word_vectors)),
                    ("gbr", GradientBoostingRegressor(n_estimators=100))])
rfr_w2v = Pipeline([("w2v_vectorizer", MeanEmbeddingVectorizer(word_vectors)),
                    ("rfr", RandomForestRegressor(n_estimators=100))])
rfr_w2v_tfidf = Pipeline([("tfidf_w2v_vectorizer", TfidfMeanVectorizer(word_vectors)),
                    ("rfr", RandomForestRegressor(n_estimators=100))])
etr_w2v = Pipeline([("w2v_vectorizer", MeanEmbeddingVectorizer(word_vectors)),
                    ("etr", ExtraTreesRegressor(n_estimators=100))])
etr_w2v_tfidf = Pipeline([("tfidf_w2v_vectorizer", TfidfMeanVectorizer(word_vectors)),
                    ("etr", ExtraTreesRegressor(n_estimators=100))])

In [ ]:
w2v_rmse_2 = sorted([(name, cv_rmse(model, X, luseful, cv=5)) 
                     for name, model in [("gbr_w2v", gbr_w2v), ("gbr_w2v_tfidf", gbr_w2v_tfidf),
                                         ("etr_w2v", etr_w2v), ("etr_w2v_tfidf", etr_w2v_tfidf),
                                         ("rfr_w2v", rfr_w2v), ("rfr_w2v_tfidf", rfr_w2v_tfidf)]], 
                  key=lambda x:x[1])

print (tabulate(w2v_rmse_2, floatfmt=".4f", headers=("model", "RMSE_5cv")))

In [31]:
# Stochastic Gradient Descent is useful for large-scale problems

sgd_w2v = Pipeline([("w2v_vectorizer", MeanEmbeddingVectorizer(word_vectors)),
                    ("sgd", SGDRegressor())])
sgd_w2v_tfidf = Pipeline([("tfidf_w2v_vectorizer", TfidfMeanVectorizer(word_vectors)),
                    ("sgd", SGDRegressor())])

# Extreme Gradient Boosting
xgb_w2v = Pipeline([("w2v_vectorizer", MeanEmbeddingVectorizer(word_vectors)),
                    ("xgb", XGBRegressor())])
xgb_w2v_tfidf = Pipeline([("tfidf_w2v_vectorizer", TfidfMeanVectorizer(word_vectors)),
                    ("xgb", XGBRegressor())])

In [ ]:
w2v_rmse_3 = sorted([(name, cv_rmse(model, X, luseful, cv=5))
                     for name, model in [("sgd_w2v", sgd_w2v), ("sgd_w2v_tfidf", sgd_w2v_tfidf),
                                         ("xgb_w2v", xgb_w2v), ("xgb_w2v_tfidf", xgb_w2v_tfidf)]], 
                  key=lambda x:x[1])

w2v_rmse = sorted(w2v_rmse_1 + w2v_rmse_2 + w2v_rmse_3, key=lambda x:x[1])

print (tabulate(w2v_rmse, floatfmt=".4f", headers=("model", "RMSE_5cv")))

w2v_models = [("lr_w2v", lr_w2v), ("lr_w2v_tfidf", lr_w2v_tfidf),
              ("lasso_w2v", lasso_w2v), ("lasso_w2v_tfidf", lasso_w2v_tfidf),
              ("ridge_w2v", ridge_w2v), ("ridge_w2v_tfidf", ridge_w2v_tfidf),
              ("gbr_w2v", gbr_w2v), ("gbr_w2v_tfidf", gbr_w2v_tfidf),
              ("etr_w2v", etr_w2v), ("etr_w2v_tfidf", etr_w2v_tfidf),
              ("rfr_w2v", rfr_w2v), ("rfr_w2v_tfidf", rfr_w2v_tfidf),
              ("sgd_w2v", sgd_w2v), ("sgd_w2v_tfidf", sgd_w2v_tfidf),
              ("xgb_w2v", xgb_w2v), ("xgb_w2v_tfidf", xgb_w2v_tfidf)]

w2v_rmse = sorted([(name, np.sqrt(np.mean(np.multiply(cross_val_score(
    model, X, luseful, cv=5, scoring='neg_mean_squared_error'), -1)))) 
                     for name, model in w2v_models], key=lambda x:x[1])

print (tabulate(w2v_rmse, floatfmt=".4f", headers=("model", "RMSE_5cv")))

### Improving XGBoost Model

In [33]:
xgb = XGBRegressor().fit(MeanEmbeddingVectorizer(word_vectors).fit(X, luseful).transform(X), luseful)

In [34]:
# Find the predicted usefulness of each word
words = np.array(list(word_vectors.keys()))
predicted = xgb.predict(list(word_vectors.values()))
indices = np.argsort(predicted)

In [38]:
useful_words = words[indices[-10:]]
useful_pred = predicted[indices[-10:]]
df = pd.DataFrame({'Most Useful Words': useful_words, 'Predicted Useful Votes': np.exp(useful_pred)-1})
print (tabulate(df, headers=df.columns, showindex=False))

Most Useful Words      Predicted Useful Votes
-------------------  ------------------------
chiropractor                         0.924456
bike                                 0.927857
someone                              0.934636
squeeze                              0.947371
hurry                                0.947432
fiance                               0.9875
doc                                  1.00934
member                               1.01952
someone_else                         1.06306
favor                                1.07591


In [37]:
useless_words = words[indices[:10]]
useless_pred = predicted[indices[:10]]
df = pd.DataFrame({'Least Useful Words': useless_words, 'Predicted Useful Votes': np.exp(useless_pred)-1})
print (tabulate(df, headers=df.columns, showindex=False))

Least Useful Words      Predicted Useful Votes
--------------------  ------------------------
carefully                           -0.103078
ail                                 -0.0926852
drastic                             -0.0876209
cellulite                           -0.0873656
horrified                           -0.0813292
disturbing                          -0.0799358
another_reviewer                    -0.0794325
content                             -0.0757281
testosterone                        -0.0736988
brush_off                           -0.072968


### Predictive Modeling with Doc2Vec Features

In [ ]:
# Gensim's Doc2Vec class creates vector representations for entire documents
# The input object is an iterator of LineSentence objects
# The default dm=1 refers to the distributed memory algorithm
# The algorithm runs through sentences twice: (1) build the vocab, 
# (2) learn a vector representation for each word and for each label (sentence)
# Better results can be achieved by iterating over the data several times

from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
doc2vec_filepath = os.path.join(intermediate_directory, 'doc2vec_model')

In [ ]:
labels = ['Review_' + str(x) for x in range(len(X))]

In [ ]:
class DocIterator():
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=doc.split(), tags=[self.labels_list[idx]])

In [ ]:
labeled_reviews = DocIterator(X, labels)

In [ ]:
# To produce better results, iterate over the data 10 times
# and control the learning rate for each iteration
if 1 == 1:
    doc2vec = Doc2Vec(size=200, window=8, min_count=5, workers=4, iter=10)
    doc2vec.build_vocab(labeled_reviews)

    doc2vec.train(labeled_reviews, total_examples=doc2vec.corpus_count, epochs=10)
    
    doc2vec.save(doc2vec_filepath)
    doc2vec.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
    
doc2vec = Doc2Vec.load(doc2vec_filepath)

In [ ]:
doc2vec.most_similar(positive=["pediatrician"], topn=5)

In [ ]:
doc2vec.most_similar(positive=['pediatrician','tooth'], negative=['baby'], topn=5)

In [ ]:
doc2vec.docvecs.most_similar('Review_10', topn=5)

In [ ]:
lr_d2v = Pipeline([("lr", LinearRegression())])
ridge_d2v = Pipeline(["ridge", Ridge(alpha=1)])
gbr_d2v = Pipeline([("gbr", GradientBoostingRegressor(n_estimators=100))])
rfr_d2v = Pipeline([("rfr", RandomForestRegressor(n_estimators=100))])
xgb_d2v = Pipeline(["xgb", XGBRegressor(n_estimators=100)])

In [ ]:
d2v_models = [("lr_d2v", lr_d2v), ("ridge_d2v", ridge_d2v) ("gbr_d2v", gbr_d2v), 
              ("rfr_d2v", rfr_d2v), ("xgb_d2v", xgb_d2v)]
d2v_rmse = [(name, np.sqrt(np.mean(np.multiply(cross_val_score(
    model, np.array(doc2vec.docvecs), luseful, cv=5, scoring='neg_mean_squared_error'), -1)))) 
              for name, model in d2v_models]

In [ ]:
rmse = sorted(w2v_rmse + d2v_rmse, key=lambda x:x[1])

print (tabulate(rmse, floatfmt=".4f", headers=("model", "RMSE_5cv")))

### Predicting Review Usefulness Based on the Amount of Training Data

In [ ]:
def benchmark(model, X, y, n):
    ss = ShuffleSplit(n_splits=5, test_size = 1 - n, random_state=0)
    scores = []
    for train, test in ss.split(X,y):
        scores.append(RMSE(y[test], model.fit(X[train],y[train]).predict(X[test])))
    return np.mean(scores)

In [ ]:
train_sizes = [0.3, 0.5, 0.7]
table = []
'''for name, model in w2v_models:
    for n in train_sizes:
        table.append({'model': name, 
                      'RMSE': benchmark(model, np.array(X), luseful, n),
                      'train_size': n})'''
for name, model in d2v_models:
    for n in train_sizes:
        table.append({'model': name, 
                      'RMSE': benchmark(model, np.array(doc2vec.docvecs), luseful, n),
                      'train_size': n})
df = pd.DataFrame(table)

In [ ]:
import seaborn as sns
plt.figure(figsize=(12,5))
fig = sns.pointplot(x='train_size', y='RMSE', hue='model',
                   data=df)#[df.model.map(lambda x: x in ['gbr_w2v_tfidf','gbr_d2v', 'etr_w2v_tfidf', 'etr_d2v'])])
sns.set_context('notebook', font_scale=1.5)
fig.set(ylabel='RMSE')
fig.set(xlabel='Training Size')
fig.set(title='Model Comparison By Training Size')
plt.show()